In [1]:
# Start Ollama
import subprocess

command = f"echo april | sudo -S systemctl start ollama"
subprocess.run(command, shell=True, check=True)

[sudo] password for aprilia: 

CompletedProcess(args='echo april | sudo -S systemctl start ollama', returncode=0)

In [2]:
import logging
logging.basicConfig(level=logging.ERROR)

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.llms import Ollama
from langchain import PromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser
import textwrap

output_parser = CommaSeparatedListOutputParser()

def load_csv_data(file_path):
    loader = CSVLoader(
        file_path=file_path,
        csv_args={"delimiter": ",", "quotechar": '"', "fieldnames": ["Prompt"]},
    )
    return loader.load()


def split_docs(documents, chunk_size=1000):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=0
    )
    chunks = text_splitter.split_documents(documents=documents)
    return chunks


def load_embedding_model(model_path, normalize_embedding=True):
    return HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs={"device": "cuda"},
        encode_kwargs={"normalize_embeddings": normalize_embedding},
    )


def create_embeddings(chunks, embedding_model, load_local, storing_path="vectorstore"):
    if load_local:
        vectorstore = FAISS.load_local(storing_path, embedding_model)
        return vectorstore

    vectorstore = FAISS.from_documents(chunks, embedding_model)
    vectorstore.save_local(storing_path)
    return vectorstore


In [4]:
from langchain.llms import Ollama
from langchain import PromptTemplate

llm = Ollama(model="llama2-uncensored", temperature=0)
embed = load_embedding_model(model_path="all-MiniLM-L6-v2")

docs = load_csv_data(file_path="lib/datasets/sd_prompts_2m_updated.csv")
documents = split_docs(documents=docs)

vectorstore = create_embeddings(chunks=documents, embedding_model=embed, load_local=True)

retriever = vectorstore.as_retriever()

/home/aprilia/anaconda3/envs/deeplearning/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-28 11:11:06.584037: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-28 11:11:06.889611: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-28 11:11:07.550101: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [5]:
from story import story

In [6]:
import spacy
from claucy import add_to_pipe
from strsimpy.levenshtein import Levenshtein

nlp = spacy.load("en_core_web_lg")
add_to_pipe(nlp)

doc = nlp(story)
lev = Levenshtein()

clauses = []

def filter_dissimilar(sentences, similarity_threshold):
    n = len(sentences)
    dissimilar_sentences = []

    for i in range(n):
        is_dissimilar = True
        for j in range(n):
            if i != j:
                similarity_score = lev.distance(sentences[i], sentences[j])

                if similarity_score >= similarity_threshold:
                    is_dissimilar = False
                    break

        if is_dissimilar:
            dissimilar_sentences.append(sentences[i])

    return dissimilar_sentences

for sentence in doc.sents:
    if len(sentence._.clauses):
        try: 
            propositions = sentence._.clauses[0].to_propositions(as_text=True)
            clauses.append(filter_dissimilar(propositions, 50))
        except:
            pass

print(clauses[0:2])


[['chaos dominated the scene', 'chaos dominated the scene \n In the heart of Gotham City'], ['The notorious Joker struck fear into the hearts of the citizens']]


In [7]:
template = """
### Task:
Generate a concise text-to-image prompt by creatively interpreting the given sentence, drawing inspiration from the provided sample prompts. Ensure that the generated prompt aligns closely with the context and meaning of the user's sentence.

### Sample Prompts for Inspiration:
{sample_prompts}

### User's Sentence:
{sentence}

### Response:

"""

prompt = PromptTemplate(template=template, input_variables=[])
llm_chain = LLMChain(llm=llm, prompt=prompt)
def getResponse(vectorstore, question):
    prompts = vectorstore.similarity_search(question)
    result = llm_chain({"sample_prompts": prompts, "sentence": sentence})
    return result["text"]

In [8]:
prompts = []

for clause in clauses:
    temp = []
    for c in clause:
        output = getResponse(vectorstore=vectorstore, question=c)
        temp.append(output)

    prompts.append(temp)

print(prompt[0:2])

chaos dominated the scene A world of chaos, Gotham City slowly returning to its uneasy calm, the memory of Margot Robbie and the Joker's twisted dance lingering, a tale of choices whispered about for years to come.
chaos dominated the scene 
 In the heart of Gotham City Prompt: Gotham City gradually returned to its uneasy calm, the memory of Margot Robbie and the Joker's twisted dance lingering, a tale of chaos and choices whispered about for years to come.
The notorious Joker struck fear into the hearts of the citizens The Joker in Gotham City, a story of chaos and choices, is etched into the city's history forever. The twisted dance between Margot Robbie and the Joker will be whispered about for years to come, leaving an indelible mark on the city's psyche.
Margot received a cryptic invitation to a concealed masquerade ball A bottle full of liquid Margot Robbie, with her eyes closed and a smile on her face, stands in the middle of a dark room, surrounded by shadows. The lighting is s

In [9]:
# Stop Ollama
command = "echo april | sudo -S systemctl stop ollama"
subprocess.run(command, shell=True, check=True)

[sudo] password for aprilia: 

CompletedProcess(args='echo april | sudo -S systemctl stop ollama', returncode=0)

In [14]:
import pybase64
import random

def getOpenposeStockImage():
    random_index = random.randint(1, 3)
    with open(f"lib/openpose/{random_index}.png", "rb") as img_file:
        encoded_img = pybase64.b64encode(img_file.read())
    return encoded_img.decode("utf-8")

def getPayloadWithLatentCouple(prompt):
    return {
        "alwayson_scripts": {
            "API payload": {"args": []},
            "Composable Lora": {
                "args": [True, False, False, False, False, False, False]
            },
            "ControlNet": {
                "args": [
                    {
                        "batch_images": "",
                        "control_mode": "Balanced",
                        "enabled": True,
                        "guidance_end": 1,
                        "guidance_start": 0,
                        "image": {"image": getOpenposeStockImage(), "mask": ""},
                        "input_mode": "simple",
                        "is_ui": True,
                        "loopback": False,
                        "low_vram": False,
                        "model": "control_sd15_openpose [fef5e48e]",
                        "module": "openpose",
                        "output_dir": "",
                        "pixel_perfect": False,
                        "processor_res": 512,
                        "resize_mode": "Crop and Resize",
                        "threshold_a": -1,
                        "threshold_b": -1,
                        "weight": 1,
                    },
                    {
                        "batch_images": "",
                        "control_mode": "Balanced",
                        "enabled": False,
                        "guidance_end": 1,
                        "guidance_start": 0,
                        "image": None,
                        "input_mode": "simple",
                        "is_ui": True,
                        "loopback": False,
                        "low_vram": False,
                        "model": "None",
                        "module": "none",
                        "output_dir": "",
                        "pixel_perfect": False,
                        "processor_res": -1,
                        "resize_mode": "Crop and Resize",
                        "threshold_a": -1,
                        "threshold_b": -1,
                        "weight": 1,
                    },
                    {
                        "batch_images": "",
                        "control_mode": "Balanced",
                        "enabled": False,
                        "guidance_end": 1,
                        "guidance_start": 0,
                        "image": None,
                        "input_mode": "simple",
                        "is_ui": True,
                        "loopback": False,
                        "low_vram": False,
                        "model": "None",
                        "module": "none",
                        "output_dir": "",
                        "pixel_perfect": False,
                        "processor_res": -1,
                        "resize_mode": "Crop and Resize",
                        "threshold_a": -1,
                        "threshold_b": -1,
                        "weight": 1,
                    },
                ]
            },
            "Extra options": {"args": []},
            "Latent Couple extension": {
                "args": [
                    True,
                    "1:1,1:2,1:2",
                    "0:0,0:0,0:1",
                    "0.2,0.8,0.8",
                    150,
                    0.2,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                ]
            },
            "OpenPose Editor": {"args": []},
            "Refiner": {"args": [False, "", 0.8]},
            "Seed": {"args": [-1, False, -1, 0, 0, 0]},
        },
        "batch_size": 1,
        "cfg_scale": 7,
        "comments": {},
        "disable_extra_networks": False,
        "do_not_save_grid": False,
        "do_not_save_samples": False,
        "enable_hr": False,
        "height": 512,
        "hr_negative_prompt": "",
        "hr_prompt": "",
        "hr_resize_x": 0,
        "hr_resize_y": 0,
        "hr_scale": 2,
        "hr_second_pass_steps": 0,
        "hr_upscaler": "Latent",
        "n_iter": 1,
        "negative_prompt": "(painting by bad-artist-anime:0.9), (painting by bad-artist:0.9), watermark, text, error, blurry, jpeg artifacts, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, artist name, (worst quality, low quality:1.4), bad anatomy (painting by bad-artist-anime:0.9), (painting by bad-artist:0.9), watermark, text, error, blurry, jpeg artifacts, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, artist name, (worst quality, low quality:1.4), bad anatomy, sexy, naked, thighs,",
        "override_settings": {"sd_model_checkpoint": "juggernaut_aftermath"},
        "override_settings_restore_afterwards": True,
        "prompt": prompt,
        "restore_faces": False,
        "s_churn": 0.0,
        "s_min_uncond": 0.0,
        "s_noise": 1.0,
        "s_tmax": None,
        "s_tmin": 0.0,
        "sampler_name": "DPM++ 2M Karras",
        "script_args": [],
        "script_name": None,
        "seed": -1,
        "seed_enable_extras": True,
        "seed_resize_from_h": -1,
        "seed_resize_from_w": -1,
        "steps": 20,
        "styles": [],
        "subseed": -1,
        "subseed_strength": 0,
        "tiling": False,
        "width": 768,
    }


def getPayload(prompt):
    return {
        "prompt": prompt,
        "negative_prompt": "(painting by bad-artist-anime:0.9), (painting by bad-artist:0.9), watermark, text, error, blurry, jpeg artifacts, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, artist name, (worst quality, low quality:1.4), bad anatomy (painting by bad-artist-anime:0.9), (painting by bad-artist:0.9), watermark, text, error, blurry, jpeg artifacts, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, artist name, (worst quality, low quality:1.4), bad anatomy, sexy, naked, thighs,",
        "override_settings_restore_afterwards": True,
        "override_settings": {"sd_model_checkpoint": "juggernaut_aftermath"},
        "width": 512,
        "height": 512
    }

In [ ]:
import os
import textwrap
import requests
from PIL import Image, ImageDraw, ImageFont
import base64
import io
import json
import random

def include_chars(sentence):
    joker_present = False
    margot_present = False
    if "joker" in sentence.lower():
        joker_present = True
    if "margot" in sentence.lower():
        margot_present = True
    if joker_present and margot_present and random.random() < 0.5:
        sentence = f"{sentence} <lora:Joker_Heath_Ledger-KK77-V1:1>,Joker,Green_hair, (masterpiece:1.0),(best_quality:1.0), ultra high res,4K,ultra-detailed,photography, 8K, HDR, highres, absurdres:1.2, Kodak portra 400, film grain, blurry background, bokeh:1.2, lens flare, (vibrant_color:1.2)  margotrobbie <lora:margotrobbie_v2:0.1>, (highly detailed skin), posing in a European street, sunset, photographic portrait, gorgeous, glamourous , classy, necklace, (modern blue dress:1.1), smile"
        return (sentence, True)
    if joker_present:
        return (f"{sentence} <lora:Joker_Heath_Ledger-KK77-V1:1>", False)
    if margot_present:
        return (f"{sentence} <lora:margotrobbie_v2:0.1>", False)

count = 0

for i, prompt in enumerate(prompts):
    for j, p in enumerate(prompt):
        result = include_chars(p)
        p = result[0]
        need_lc = result[1]
        payload = None
        if need_lc:
            payload = getPayloadWithLatentCouple(p)
        else:
            payload = getPayload(p)

        response = requests.post(url="http://127.0.0.1:7860/sdapi/v1/txt2img", json=payload)
        data = response.json()
        responseImage = data["images"][0]
        image = Image.open(io.BytesIO(base64.b64decode(responseImage.split(",", 1)[0])))
        res_image = Image.new("RGB", (image.width, image.height + 80), "white")
        res_image.paste(image, (0, 0))
        draw = ImageDraw.Draw(res_image)
        wrapped_text = textwrap.fill(story[i], width=(image.width / 8) - 20)
        draw.text(
        xy=(10, image.height + 10),
        text=wrapped_text,
        fill=(0, 0, 0),
        font=ImageFont.truetype(
            font=r"/home/aprilia/.local/share/fonts/CaskaydiaCoveNerdFontMono-Regular.ttf",
            size=16,
        ),
        )
        res_image.save(f"dumps3/sample{count}.png")
        print(f"Saved {count}")
        count += 1